<a href="https://colab.research.google.com/github/brucecmd/tensorflow_in_action_code/blob/master/%E5%88%9D%E6%AD%A5%E7%9A%84%E5%85%B8%E5%9E%8B%E4%BE%8B%E5%AD%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [0]:
input_node = 784
output_node = 10
layer1_node = 500

In [0]:
def get_weight_variable(shape, regularizer):
  weights = tf.get_variable("weights", shape, initializer=tf.truncated_normal_initializer(stddev=0.1))
  if regularizer!=None:
    tf.add_to_collection("losses", regularizer(weights))
  return weights

In [0]:
def inference(input_tensor, regularizer):
  with tf.variable_scope("layer1"):
    weights = get_weight_variable([input_node, layer1_node], regularizer)
    biases = tf.get_variable("biases", [layer1_node], initializer=tf.constant_initializer(0.0))
    layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)
  with tf.variable_scope("layer2"):
    weights = get_weight_variable([layer1_node,output_node], regularizer)
    biases = tf.get_variable("biases", [output_node], initializer=tf.constant_initializer(0.0))
    layer2 = tf.matmul(layer1, weights) + biases
  return layer2

In [0]:
import os
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
batch_size = 100
learning_rate_base = 0.8 # learning_rate会decay, 这个是最初的learning_rate
learning_rate_decay = 0.99  # learning_rate decay的参数
regularization_rate = 0.0001 # 对weights进行正则项的参数
training_steps = 30000 # 一共训多少次，这个不是epoch，而是每一个batch_size算一次
moving_average_decay = 0.99 # 参与训练的参数的滑动平均参数

model_save_path = "/content/drive/My Drive/model_folder/test_model4/"
model_name = "model.ckpt"

In [7]:

from google.colab import drive
drive.mount('/content/drive')  # 这一步是连接google drivea

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
import os
print(os.path.exists("/content/drive/My Drive/model_folder/test_model3/"))
print(os.path.exists("/content/drive/My Drive/model_folder/test_model4/"))

True
False


In [0]:
def train(mnist):
  x = tf.placeholder(tf.float32, [None, input_node], name="x-input")
  y_ = tf.placeholder(tf.float32, [None, 10], name="y-input")
  
  regularizer = tf.contrib.layers.l2_regularizer(regularization_rate)
  
  y = inference(x, regularizer)
  
  global_step = tf.Variable(0, trainable=False)
  variables_averages = tf.train.ExponentialMovingAverage(moving_average_decay, global_step)
  variables_averages_op = variables_averages.apply(tf.trainable_variables())
  
  cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.argmax(y_,1), logits=y)
  cross_entropy_mean = tf.reduce_mean(cross_entropy)
  loss = cross_entropy_mean + tf.add_n(tf.get_collection("losses"))

  learning_rate = tf.train.exponential_decay(learning_rate_base, global_step, mnist.train.num_examples/batch_size, learning_rate_decay)
  
  train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  with tf.control_dependencies([train_step, variables_averages_op]):
    train_op = tf.no_op(name="train")
    
  saver = tf.train.Saver()
  
  with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for i in range(training_steps):
      xs, ys = mnist.train.next_batch(batch_size)
      _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x:xs, y_:ys})
      
      if i % 1000 == 0:
        print("After %d training steps, loss on training batch is %g"%(step, loss_value))
        
        # 训练1000轮之后得到的模型
        # 注意，保存模型的时候，还给出了训练的轮数，也就是global_step参数，这样，保存出来的模型就是类似于model.ckpt-1000这样的；
        saver.save(sess, os.path.join(model_save_path, model_name), global_step=global_step)

In [13]:
mnist = input_data.read_data_sets("./MNIST", one_hot=True)

Extracting ./MNIST/train-images-idx3-ubyte.gz
Extracting ./MNIST/train-labels-idx1-ubyte.gz
Extracting ./MNIST/t10k-images-idx3-ubyte.gz
Extracting ./MNIST/t10k-labels-idx1-ubyte.gz


In [14]:
print(mnist.train.num_examples)

55000


In [15]:
train(mnist)

W0901 03:25:45.995540 140274740156288 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/moving_averages.py:433: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


After 1 training steps, loss on training batch is 3.57218
After 1001 training steps, loss on training batch is 0.223444
After 2001 training steps, loss on training batch is 0.149209
After 3001 training steps, loss on training batch is 0.162809
After 4001 training steps, loss on training batch is 0.1277


W0901 03:26:16.466108 140274740156288 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


After 5001 training steps, loss on training batch is 0.142235
After 6001 training steps, loss on training batch is 0.0959694
After 7001 training steps, loss on training batch is 0.0862885
After 8001 training steps, loss on training batch is 0.0868254
After 9001 training steps, loss on training batch is 0.0780453
After 10001 training steps, loss on training batch is 0.0672748
After 11001 training steps, loss on training batch is 0.0638042
After 12001 training steps, loss on training batch is 0.0610936
After 13001 training steps, loss on training batch is 0.0520225
After 14001 training steps, loss on training batch is 0.0513625
After 15001 training steps, loss on training batch is 0.0545524
After 16001 training steps, loss on training batch is 0.0446302
After 17001 training steps, loss on training batch is 0.0523867
After 18001 training steps, loss on training batch is 0.0435585
After 19001 training steps, loss on training batch is 0.0457951
After 20001 training steps, loss on training b